In [132]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
import time

# Задание 1

### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [2]:
# Создаём функцию, которая собирает с сайта habr.com/ru ссылки на все посты материалы (статьи, посты),
# в которых содержится указанное ключевое слово

def get_all_links(query, pages):
    
    links = []
    
    for page in range(1, pages+1):
        
        URL = 'https://habr.com/ru/search/' + 'page' + str(page)
        params = {
        'q': str(query)
        }

        res = requests.get(URL, params)
        soup = BeautifulSoup(res.text, 'html.parser')
        articles = soup.find_all('article')
        list_of_links = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), articles))
        links.extend(list_of_links)
        
    return links

In [3]:
%%time

# Применяем данную функцию для ключевых слов из списка keywords

keywords = ['python', 'парсинг']

links = []

for keyword in keywords:
    links.extend(get_all_links(keyword, 50))
    
links

Wall time: 1min 33s


['https://habr.com/ru/news/t/531402/',
 'https://habr.com/ru/news/t/498364/',
 'https://habr.com/ru/company/itsumma/news/t/490834/',
 'https://habr.com/ru/news/t/505096/',
 'https://habr.com/ru/news/t/527858/',
 'https://habr.com/ru/news/t/473926/',
 'https://habr.com/ru/news/t/519414/',
 'https://habr.com/ru/company/piter/blog/528880/',
 'https://habr.com/ru/post/147281/',
 'https://habr.com/ru/post/205944/',
 'https://habr.com/ru/company/otus/blog/425233/',
 'https://habr.com/ru/company/mailru/blog/454324/',
 'https://habr.com/ru/post/484446/',
 'https://habr.com/ru/company/intersystems/blog/486984/',
 'https://habr.com/ru/post/161931/',
 'https://habr.com/ru/post/168083/',
 'https://habr.com/ru/post/168827/',
 'https://habr.com/ru/post/169639/',
 'https://habr.com/ru/post/203516/',
 'https://habr.com/ru/company/mailru/blog/234747/',
 'https://habr.com/ru/company/mailru/blog/254727/',
 'https://habr.com/ru/post/269411/',
 'https://habr.com/ru/company/mailru/blog/337364/',
 'https://h

In [4]:
%%time

# и теперь в цикле для каждой ссылки из списка извлекаем и добавляем в датафрейм искомую информацию:

df = pd.DataFrame()

for link in links:
    
    request = requests.get(link)
    link_soup = BeautifulSoup(request.text, 'html.parser')
    
    # опытным путём обнаружено, что дата и заголовок хранятся в словаре, который можно спасрсить следующим образом:    
    
    article_dict = json.loads(''.join(link_soup.find('script', {'type': 'application/ld+json'}).contents))
    
    date = datetime.strptime(article_dict['datePublished'], '%Y-%m-%dT%H:%M:%S%z')
    headline = article_dict['headline']
    
    # опытным путём обнаружено, что текст статьи на страницах размечен по-разному, поэтому его парсим по-разному:
    
    text = []
    
    if link_soup.find('div', class_ = 'post__text post__text_v1'):
        text = link_soup.find('div', class_ = 'post__text post__text_v1').text
    
    if link_soup.find('div', class_ = 'post__text post__text_v2'):
        text = link_soup.find('div', class_ = 'post__text post__text_v2').text
        
    if link_soup.find('div', class_ = 'post__text post__text-html post__text_v1'):
        text = link_soup.find('div', class_ = 'post__text post__text-html post__text_v1').text
        
    row = {'post_date': date, 
           'headline': headline, 
           'link': link, 
           'text': text }
    
    df = pd.concat([df, pd.DataFrame([row])])
    
df

Wall time: 31min 47s


,post_date,headline,link,text
0,2020-12-04 21:03:25+03:00,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/,По данным широко известного в узких кругах Tio...
0,2020-04-21 18:35:14+03:00,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"\r\n20 апреля 2020 года, спустя почти десять л..."
0,2020-03-03 13:22:32+03:00,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P..."
0,2020-06-03 09:00:30+03:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/t/505096/,\n\r\n31 мая 2020 года разработчик Джош Мур (J...
0,2020-11-13 10:10:45+03:00,Создатель Python Гвидо ван Россум выходит на р...,https://habr.com/ru/news/t/527858/,\n\r\nСоздатель языка программирования Python ...
...,...,...,...,...
0,2013-11-13 21:07:38+04:00,Как я делал свой фотошеринг с геолокацией и бл...,https://habr.com/ru/company/flatmania/blog/202...,\n\nГонки на яхтах и пристрастие к наркотикам ...
0,2013-11-19 12:28:46+04:00,Миграция приложения с Windows 8 на Windows 8.1,https://habr.com/ru/company/microsoft/blog/202...,Миграция приложений на новую версию операционн...
0,2013-11-21 15:23:35+04:00,Беспроводные коммуникации «умного дома». Часть...,https://habr.com/ru/post/202898/,После написания пары больших постов про «радио...
0,2013-11-24 20:02:39+04:00,Пример реализации истории звонков для IP-АТС 3...,https://habr.com/ru/company/3cx/blog/203454/,Предлагаем вам взглянуть на пример реализации ...


# Задание 2

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [174]:
emails = ['e_goun@belmag.ru', 'ypa_gun@mail.ru']

In [182]:
df_leaks = pd.DataFrame()

for email in emails:
    
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

    params = { 'emailAddresses': [email] }
    
    headers = {'Vaar-Version': '0',
               'Vaar-Header-App-Product': 'hackcheck-web-avast',
               'Content-Type': 'application/json;charset=UTF-8',
               'Host': 'identityprotection.avast.com',
               'Origin': 'https://www.avast.com',
               'Referer': 'https://www.avast.com/',
               'vaar-header-captcha-response-token': '03AGdBq26r4LzketNJyAZwqhNn_xUvpCDVBWeqSi22LGFuHGtYXmmvuMaM0SyDB5z4ZMkSpXguHvoTssuWm_FrQeB9gaNmGcIupugGIoz1c2eJbMJXHfXnG-BS6XnldZBVWX4MkyGsDkc0wPsgjbqy8VwlBphCqP87nEbtYiXUTa6NQbpMJYlgGyHMf5sXNMvPAiXhm1SnDkQbSY9uBZ_34VSxUYgKNJJ6C65kc-qnnxWa7qJr5rcy0ofdR9tNLWQWqxiZG49joN0o3QYHjqR76XrM-SpyYue2UhILbYAKdVGg5SrwBhYWd5sPXepeV64eChO8vQc6WeDyriLBUAHvhpCuGD1yCabCtRBtYZnMur0NqGZ9pLYFlWJR8x_DwUfYPv6ARLpqM74gwjYj75mdTVjv195wSDhL9xUWKv7CsBmqaKg_MezbtXTVkSPjoSz0cu_z1L6GA4Vg' }

    req = requests.post(URL, json=params, headers=headers)
    response_soup = BeautifulSoup(req.text, 'html.parser')
    response_json = json.loads(str(response_soup))
    
    for breachID in response_json['summary'][str(email)]['breaches']:
        
        e_mail = email
        date = datetime.strptime(response_json['breaches'][str(breachID)]['publishDate'], '%Y-%m-%dT%H:%M:%S%z')
        website = response_json['breaches'][str(breachID)]['site']
        comment = response_json['breaches'][str(breachID)]['description']

        row = {'email': e_mail,
               'date': date,
               'website': website,
               'comment': comment }

        df_leaks = pd.concat([df_leaks, pd.DataFrame([row])])

df_leaks 

,email,date,website,comment
0,e_goun@belmag.ru,2016-10-21 00:00:00+00:00,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,ypa_gun@mail.ru,2016-10-25 00:00:00+00:00,tumblr.com,"In May 2016, microblogging platform Tumblr rev..."
0,ypa_gun@mail.ru,2019-05-23 00:00:00+00:00,livejournal.com,"In 2017, social network LiveJournal's database..."
